# Lecture 2. 신경망의 수학적 이해
* Ref. 최건호, 파이토치 첫걸음, 한빛미디어

In [13]:
import numpy as np 
import tensorflow as tf
import torch
from datetime import datetime

## 1. Framework Comparison

In [14]:
# 연산에 필요한 numpy, 시간을 측정하기 위해 datetime을 불러옵니다.
start = datetime.now()

# 랜덤하게 3x4 형태의 변수 x,y,z를 설정해줍니다.
np.random.seed(0)

N,D = 3,4

x = np.random.randn(N,D)
y = np.random.randn(N,D)
z = np.random.randn(N,D)

# x,y,z를 이용해 x*y+z를 계산해줍니다.
a = x * y
b = a + z
c = np.sum(b)

# 기울기(gradient)가 1이라고 가정하고 역전파를 해줍니다. 역전파에 대한 내용은 4장에서 자세히 다룹니다.
grad_c = 1.0
grad_b = grad_c * np.ones((N,D))
grad_a = grad_b.copy()
grad_z = grad_b.copy()
grad_y = grad_a * y
grad_x = grad_a * x

# 각각의 기울기가 몇인지 걸린 시간은 얼마인지 확인해봅니다.
print(grad_x)
print(grad_y)
print(grad_z)
print(datetime.now()-start)

[[ 1.76405235  0.40015721  0.97873798  2.2408932 ]
 [ 1.86755799 -0.97727788  0.95008842 -0.15135721]
 [-0.10321885  0.4105985   0.14404357  1.45427351]]
[[ 0.76103773  0.12167502  0.44386323  0.33367433]
 [ 1.49407907 -0.20515826  0.3130677  -0.85409574]
 [-2.55298982  0.6536186   0.8644362  -0.74216502]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
0:00:00.002828


In [15]:
# 이번에는 텐서플로 프레임워크를 이용해 같은 연산을 해보도록 하겠습니다.
start = datetime.now()

# 텐서플로는 연산 그래프를 먼저 정의하고 추후에 여기에 값을 전달하는 방식입니다. 여기서는 비어있는 그래프만 정의해줍니다.
# Define Graph on GPU
with tf.device('/gpu:0'):              # 해당 연산을 0번 gpu에서 하겠다는 의미입니다.
    tf.random.set_seed(0)

    N,D = 3,4
    
    x = tf.Variable(tf.random.uniform(shape=(N,D)), dtype=tf.float32)
    y = tf.Variable(tf.random.uniform(shape=(N,D)), dtype=tf.float32)
    z = tf.Variable(tf.random.uniform(shape=(N,D)), dtype=tf.float32)

    with tf.GradientTape() as tape:        # 텐서플로의 경우 계산 그래프에서 작동합니다.
        a = x * y                          # 연산 과정 또한 정의해줍니다.
        b = a + z
        c = tf.reduce_sum(b)
        [grad_x, grad_y, grad_z] = tape.gradient(c,[x,y,z])  # c에 대한 x,y,z의 기울기(gradient)를 구하고 이를 각각 grad_x, grad_y, grad_z에 저장하도록 지정해놓습니다.

# 값들을 확인하고 걸린 시간을 측정합니다.
print(grad_x)
print(grad_y)
print(grad_z)
print(datetime.now()-start)

tf.Tensor(
[[0.5554141  0.22129297 0.8649249  0.77728355]
 [0.6451167  0.53036225 0.01444101 0.87350917]
 [0.4697218  0.38672888 0.88615394 0.6367892 ]], shape=(3, 4), dtype=float32)
tf.Tensor(
[[0.29197514 0.20656645 0.53539073 0.5612575 ]
 [0.4166745  0.80782795 0.4932251  0.99812925]
 [0.69673514 0.1253736  0.7098167  0.6624156 ]], shape=(3, 4), dtype=float32)
tf.Tensor(
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]], shape=(3, 4), dtype=float32)
0:00:00.036832


In [12]:
# 이번에는 파이토치를 이용해 같은 연산을 진행해보도록 하겠습니다.
from datetime import datetime
start = datetime.now()

N,D = 3,4

# x,y,z를 랜덤하게 초기화 해줍니다. 
# https://pytorch.org/docs/stable/torch.html?highlight=randn#torch.randn

x = torch.randn(N,D,device=torch.device('cuda'), requires_grad=True)
y = torch.randn(N,D,device=torch.device('cuda'), requires_grad=True)
z = torch.randn(N,D,device=torch.device('cuda'), requires_grad=True)

# 연산 그래프는 정의됨과 동시에 연산됩니다.
a = x * y
b = a + z
c = torch.sum(b)

# 기울기(gradient)가 1.0라고 가정하고 최종 값인 c에서 backward를 통해 역전파를 해줍니다.
# 넘파이와 비교했을때 이 과정이 자동적으로 게산되는 것을 확인할 수 있습니다.
c.backward(gradient=torch.cuda.FloatTensor([1.0]))

# 각각의 기울기와 걸린 시간을 출력합니다.
print(x.grad)
print(y.grad)
print(z.grad)
print(datetime.now()-start)

RuntimeError: No CUDA GPUs are available